In [ ]:
import pandas as pd
import numpy as np
import ast

## Reading and processing data from INPE Researcher And Publications

In [ ]:
dataset_dir = '/mnt/d/Dados/OneDrive/IBICT/datasets/inpe/'
file_researchers = 'Lattes_Metadata.csv'
file_pubs = 'Lattes_Papers.csv'

In [ ]:
encoding = "ISO-8859-1"

sel_cols=['i16','year','title','journal']

def converter(val):
    if val == np.nan:
        return 0
    return val

dfr = pd.read_csv(dataset_dir+file_researchers,sep=',',encoding=encoding)
dfp = pd.read_csv(dataset_dir+file_pubs,sep=',',
                    usecols=sel_cols,
                    # dtype={"year": "int16"},
                    # chunksize=1000,
                    encoding=encoding)

In [ ]:
researchers = dfr.set_index(['i16'])
researchers.sort_index(inplace=True)

## Populate Dπ-dev net

In [ ]:
import os
import time
import logging

from tqdm.notebook import tqdm

from web3.providers.eth_tester import EthereumTesterProvider
from web3 import Web3
from web3.auto import w3

from web3.exceptions import TransactionNotFound
from eth_tester import PyEVMBackend
from hexbytes import HexBytes


from libs import compile_contract
from eth_tester.exceptions import TransactionFailed

### Methods

In [ ]:
def get_tx_params(gas,account,chain_id=1337,min_gas_price='100'):
    # nonce = w3.eth.getTransactionCount(acount.address)
    nonce = w3.eth.getTransactionCount(w3.toChecksumAddress(account.address))
    tx_params = {'from': account.address,
                # 'to': contractAddress,
                'nonce': nonce,
                'gas': gas * 2,
                'gasPrice': w3.toWei(min_gas_price, 'gwei'), # defaul min gas price to besu (nao funciona w3.eth.gas_price)
                'chainId': chain_id
    }
    return tx_params

### Execution

Network configuration

In [ ]:
# virtual env
# w3 = Web3(EthereumTesterProvider(PyEVMBackend()))

# Test Env
# HTTPProvider:
from web3 import Web3, IPCProvider
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

chain_id=1337

account importing

In [ ]:
privateKey='c87509a1c067bbde78beb793e6fa76530b6382a4c0241e5e4a9ec0a0f44dc0d3'
acount = w3.eth.account.privateKeyToAccount(privateKey)
# w3.eth.get_balance(acount.address)

Contract setup

In [ ]:
researcher_db_addr = '0xC8c03647d39a96f02f6Ce8999bc22493C290e734'
publication_db_addr = '0x446e083592D2dFa6661E20097F6898990C3CACE5'
# researcher_db_addr = '0xc0ED63E3A70BfCB003452B1Cc083db822e1f23e1'

In [ ]:
dapp_dir = '..'+os.sep+'Dapp'+os.sep+'contracts'+os.sep

dapp_libs = [ dapp_dir + 'libs' + os.sep + 'EntitiesLib.sol' , dapp_dir + 'util' +os.sep+ 'HitchensUnorderedKeySet.sol']

contract_interface = compile_contract('ResearcherDB.sol',dapp_dir,dapp_libs)
print('ResearcherDB :: ', researcher_db_addr)
researcher_db_contract = w3.eth.contract(address=researcher_db_addr, abi=contract_interface["abi"])

# PUblications
contract_interface = compile_contract('PublicationDB.sol',dapp_dir,dapp_libs)
print('PublicationDB :: ', publication_db_addr)
publication_db_contract = w3.eth.contract(address=publication_db_addr, abi=contract_interface["abi"])


populate

In [ ]:
publications = dfp

tempo_pesquisador = [['0',0,0]]
tempo_publicacao = [['0',0,0]]

ee = []
titulos = publications.title.unique()

for titulo in tqdm(titulos[14000:]):
    ldf = publications[publications.title == titulo]
    ano = ldf.year.head(1).values[0]
    
    authors = []
    #recuperando e criando os autores
    try:
        for row in ldf.itertuples():
            researcher_name = researchers.loc[row.i16]['name']
            
            exist_flag, author_id = researcher_db_contract.functions.getResearcherByName(researcher_name).call()

            est_gas = 0
            if exist_flag == False:
                start = time.time()
                #estimate de gas
                est_gas = researcher_db_contract.functions.createResearcher(researcher_name).estimateGas()
                #build the tx params
                tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)

                tx = researcher_db_contract.functions.createResearcher(researcher_name).buildTransaction(tx_params)
                signed_tx = acount.signTransaction(tx)
                # send the transaction
                tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
                receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
                end_create_researcher = time.time() - start
                debug_line = [Web3.toHex(author_id),est_gas,end_create_researcher]
                tempo_pesquisador.append(debug_line)
            #adciona a lista dos autores da publicacao
            authors.append(author_id)
        
            try:
                start = time.time()
                est_gas = publication_db_contract.functions.createPublication(titulo,int(ano),'journal',authors).estimateGas()
                tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)
                tx = publication_db_contract.functions.createPublication(titulo,int(ano),'journal',authors).buildTransaction(tx_params)
                
                signed_tx = acount.signTransaction(tx)
                tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
                receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

                end_create_pub = time.time() - start
                debug_line = [receipt['transactionHash'],est_gas,end_create_pub]
                tempo_publicacao.append(debug_line)
                time.sleep(.2)
                
            # except TransactionFailed:
            #     print('    JA EXISTE')
            except ValueError as e:
                import ast
                e_error = ast.literal_eval(str(e))
                if e_error['message'] == 'Execution reverted':
                    # print('    JA EXISTE - ' + titulo)
                    break
    except ValueError as e: 
        e_error = ast.literal_eval(str(e))
        if e_error['message'] == 'Execution reverted':
            print(e_error['message'] + '::' + e_error['data'])
            break
        if e_error['message'] == 'Transaction gas limit exceeds block gas limit':
            print(e_error['message'])

#
tempo_pesquisador = pd.DataFrame(tempo_pesquisador)
tempo_pesquisador.columns = ['researche_id','gas','time']
tempo_pesquisador.to_csv('tempo_pesquisador.csv', mode='a', index=False,header=False)

tempo_publicacao = pd.DataFrame(tempo_publicacao)
tempo_publicacao.columns = ['transactionHash','gas','time']
tempo_publicacao.to_csv('tempo_publicacao.csv', mode='a', index=False,header=False)

In [ ]:
tempo_pesquisador = pd.DataFrame(tempo_pesquisador)
tempo_pesquisador.columns = ['researche_id','gas','time']
tempo_pesquisador.to_csv('tempo_pesquisador.csv', mode='a', index=False,header=False)

tempo_publicacao = pd.DataFrame(tempo_publicacao)
tempo_publicacao.columns = ['transactionHash','gas','time']
tempo_publicacao.to_csv('tempo_publicacao.csv', mode='a', index=False,header=False)

### Other contract Iteractions

iteractive operation to the smart contracts

In [ ]:
researcher_db_contract.functions.getResearcherByName('Ze das couves').call()

In [ ]:
#estimate de gas
est_gas = researcher_db_contract.functions.createResearcher('Maria').estimateGas()
            #build the tx params
tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)

tx = researcher_db_contract.functions.createResearcher('Maria').buildTransaction(tx_params)
signed_tx = acount.signTransaction(tx)
# send the transaction
tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
researcher_db_contract.functions.getResearcherByName('Maria').call()

In [ ]:
id = researcher_db_contract.functions.getResearcherIdAtIndex(0).call()
# id = '0x13e3d7a18183146440048716dd45b252200133f27bef8a56d852de498a2f8d4a'
researcher_db_contract.functions.getResearcherById(id).call()

In [ ]:
researcher_db_contract.functions.countResearchers().call()

In [ ]:
publication_db_contract.functions.countPublications().call()

In [ ]:
idp1 = publication_db_contract.functions.getPublicationIdAtIndex(1).call()
publication_db_contract.functions.getPublicationById(idp1).call()

In [ ]:
id = b'\x18\xa1\x8a\xac\x0e\x11\xeb\x92\xa7\xb0e\xf81\x04\x90G\xf5\x1f\xb0\xd0\x88yyG\x81+#\xff\xa6\\g\xd2'
Web3.toHex(id)

In [ ]:
researcher_db_contract.functions.getResearcherById(id).call()